In [ ]:
import ipywidgets as widgets
from IPython.display import display


layout = widgets.Layout(width='50%')
style = {'description_width': 'initial'}

data_source_options = ['Upload file...', 'Local file...', 'Example: SMLMS2013_HDTubulinAlexa647']
w = {
    "data_source": widgets.Dropdown(options=data_source_options, 
        value=data_source_options[0], description='Data source:', layout=layout, style=style),
    "data_source_action": widgets.Button(description="Set data!"), 
    "ring_radius": widgets.FloatSlider(value=0.5, min=0.1, max=3.0, description='Ring Radius:', layout=layout, style=style),
    "magnification": widgets.IntSlider(value=4, min=1, max=10, description='Magnification:', layout=layout, style=style),
    "do_drift_correction": widgets.Checkbox(value=False, description='Do drift correction', layout=layout),
    "m1": widgets.Label("-=-= Time-Lapse =-=-"),
    "frames_per_timepoint": widgets.IntText(value=0, description='Frames per time-point (0 - auto)', style=style),
    "m2": widgets.Label("-=-= Crop Data =-=-"),
    "start_frame": widgets.IntText(value=0, description='Start analysis on frame (0 - auto)', style=style),
    "end_frame": widgets.IntText(value=0, description='End analysis on frame (0 - auto)', style=style),
}


def on_file_upload(change):
    # Handle the uploaded file here
    print(change)

def data_source_action(b):
    print(w['data_source'].value)
    if w['data_source'].value == 'Upload file...':
        # make the jupyter notebook display a file upload dialog
        fu = widgets.FileUpload(accept='image/*', multiple=False)
        fu.observe(on_file_upload, names='value') 
        display(fu)

w['data_source_action'].on_click(data_source_action)

display(*w.values())


In [ ]:
print(w['frames_per_timepoint'].value)